In [40]:
import numpy as np
import time
import cv2
import os

In [41]:
### numpy

In [42]:
### Nested

In [43]:
def darken_blend_8_np(base, active):
  return np.where(np.greater(base, active), active, base)

In [44]:
def color_burn_8_np(base, active):
  return np.where(np.equal(active, 0.0), 255.0, 255.0 - (255.0 - base) / active)

In [45]:
def lighten_blend_8_np(base, active):
  return np.where(np.less(base, active), active, base)

In [46]:
def color_dodge_8_np(base, active):
  return np.where(np.equal(active, 255.0), 255.0, base / (255.0 - active))

In [47]:
def overlay_blend_8_np(base, active):
  return np.where(np.greater_equal(base, 128.0), 2 * base + base - 2 * base * base / 255.0 - 128.0, 2 * base * base / 128.0)

In [48]:
def multiply_blend_8_np(base, active):
  return base * active / 255.0

In [49]:
def linear_burn_8_np(base, active):
  return base + active - 255.0

In [50]:
def screen_blend_8_np(base, active):
  return base + active - base * active / 255.0

In [51]:
def linear_dodge_8_np(base, active):
  return base + active

In [52]:
### Single

In [53]:
def normal_blend_f_np(base, active, opacity):
  return opacity * active + (1-opacity)*base

In [54]:
def normal_blend_8_np(base, active, opacity):
  return opacity * active + (255.0 - opacity) * base

In [55]:
# Runner

In [56]:
rng = np.random.default_rng(1)

In [57]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    rnd = rng.random(img.shape, dtype = np.float32) * 255
    bases.append(img)
    actives.append(rnd)

In [58]:
def mat_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = bases[i]
        a = actives[i]
        start_time = time.perf_counter()
        f(b, a)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def vec_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = bases[i].flatten()
        a = actives[i].flatten()
        opacity = rng.random(1, dtype = np.float32)
        start_time = time.perf_counter()
        f(b, a, opacity)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def timer(bases, actives, f, runner):
    runs = 5
    times = []
    for _ in range(runs):
        times.append(runner(bases, actives, f))
    times = np.array(times)
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [59]:
timer(bases, actives, darken_blend_8_np, mat_runner)

4900.796507066116ms +/- 14.689765803660851ms


In [60]:
timer(bases, actives, color_burn_8_np, mat_runner)

/tmp/ipykernel_2457541/2120156089.py:2: RuntimeWarning: divide by zero encountered in divide
  return np.where(np.equal(active, 0.0), 255.0, 255.0 - (255.0 - base) / active)
/tmp/ipykernel_2457541/2120156089.py:2: RuntimeWarning: invalid value encountered in divide
  return np.where(np.equal(active, 0.0), 255.0, 255.0 - (255.0 - base) / active)


4601.456473628059ms +/- 52.35595380857583ms


In [61]:
timer(bases, actives, lighten_blend_8_np, mat_runner)

4907.451175572351ms +/- 26.818089681326725ms


In [62]:
timer(bases, actives, color_dodge_8_np, mat_runner)

2059.3938098289073ms +/- 10.74560826905346ms


In [63]:
timer(bases, actives, overlay_blend_8_np, mat_runner)

7276.276847813278ms +/- 38.89864636360789ms


In [64]:
timer(bases, actives, multiply_blend_8_np, mat_runner)

955.734866252169ms +/- 15.603476675136935ms


In [65]:
timer(bases, actives, linear_burn_8_np, mat_runner)

822.7266173344105ms +/- 12.299932230623227ms


In [66]:
timer(bases, actives, screen_blend_8_np, mat_runner)

1804.7624012455344ms +/- 14.057950391944754ms


In [67]:
timer(bases, actives, linear_dodge_8_np, mat_runner)

659.931408520788ms +/- 3.539213676314929ms


In [68]:
timer(bases, actives, normal_blend_f_np, vec_runner)

1050.687387259677ms +/- 1.7066460054735344ms


In [69]:
timer(bases, actives, normal_blend_8_np, vec_runner)

1048.7009353470057ms +/- 16.85136312652299ms
